<h1><center>Implementando Transacciones

<h2>Parte 01. Transacciones y el motor de base de datos

1.1. Abrir la base de datos TSQL.

In [1]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00.181

1.2. Procedemos a crear dos tablas para las demostraciones

In [2]:
IF OBJECT_ID('dbo.SimpleOrderDetails','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrderDetails;
IF OBJECT_ID('dbo.SimpleOrders','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrders;
GO
CREATE TABLE dbo.SimpleOrders(
	orderid int IDENTITY(1,1) NOT NULL PRIMARY KEY,
	custid int NOT NULL FOREIGN KEY REFERENCES Sales.Customers(custid),
	empid int NOT NULL FOREIGN KEY REFERENCES HR.Employees(empid),
	orderdate datetime NOT NULL
);
GO
CREATE TABLE dbo.SimpleOrderDetails(
	orderid int NOT NULL FOREIGN KEY REFERENCES dbo.SimpleOrders(orderid),
	productid int NOT NULL FOREIGN KEY REFERENCES Production.Products(productid),
	unitprice money NOT NULL,
	qty smallint NOT NULL,
 CONSTRAINT PK_OrderDetails PRIMARY KEY (orderid, productid)
);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.499

1.3. Ahora ejecutamos multiples setencias de manipulaci{on de datos contra las tablas recién creadas. (Se producira un Error)

In [3]:
BEGIN TRY
	INSERT INTO dbo.SimpleOrders(custid, empid, orderdate) VALUES (68,9,'2006-07-12');
	INSERT INTO dbo.SimpleOrders(custid, empid, orderdate) VALUES (88,3,'2006-07-15');
	INSERT INTO dbo.SimpleOrderDetails(orderid,productid,unitprice,qty) VALUES (1, 2,15.20,20);
	INSERT INTO dbo.SimpleOrderDetails(orderid,productid,unitprice,qty) VALUES (999,77,26.20,15);
END TRY
BEGIN CATCH
	SELECT ERROR_NUMBER() AS ErrNum, ERROR_MESSAGE() AS ErrMsg;
END CATCH;

(1 row affected)

(1 row affected)

(1 row affected)

(0 rows affected)

(1 row affected)

Total execution time: 00:00:00.602

ErrNum,ErrMsg
547,"The INSERT statement conflicted with the FOREIGN KEY constraint ""FK__SimpleOrd__order__5EBF139D"". The conflict occurred in database ""TSQL"", table ""dbo.SimpleOrders"", column 'orderid'."


1.4. Verificamos que inclusive con una excepción, parcialmente se han insertado ciertos registros

In [4]:
SELECT  orderid, custid, empid, orderdate
FROM dbo.SimpleOrders;
SELECT  orderid, productid, unitprice, qty
FROM dbo.SimpleOrderDetails;

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.252

orderid,custid,empid,orderdate
1,68,9,2006-07-12 00:00:00.000
2,88,3,2006-07-15 00:00:00.000


orderid,productid,unitprice,qty
1,2,15.2000,20


1.5. Finalizamos limpiando los objetos creados.

In [5]:
IF OBJECT_ID('dbo.SimpleOrderDetails','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrderDetails;
IF OBJECT_ID('dbo.SimpleOrders','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrders;

Commands completed successfully.

Total execution time: 00:00:00.147

<h2>Parte 02. Controlando transacciones.

2.1. Creamos las tablas para la demostración.

In [13]:
IF OBJECT_ID('dbo.SimpleOrderDetails','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrderDetails;
IF OBJECT_ID('dbo.SimpleOrders','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrders;
GO
CREATE TABLE dbo.SimpleOrders(
	orderid int IDENTITY(1,1) NOT NULL PRIMARY KEY,
	custid int NOT NULL FOREIGN KEY REFERENCES Sales.Customers(custid),
	empid int NOT NULL FOREIGN KEY REFERENCES HR.Employees(empid),
	orderdate datetime NOT NULL
);
GO
CREATE TABLE dbo.SimpleOrderDetails(
	orderid int NOT NULL FOREIGN KEY REFERENCES dbo.SimpleOrders(orderid),
	productid int NOT NULL FOREIGN KEY REFERENCES Production.Products(productid),
	unitprice money NOT NULL,
	qty smallint NOT NULL,
 CONSTRAINT PK_OrderDetails PRIMARY KEY (orderid, productid)
);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.531

2.2. Crear una transacción con varias instrucciones INSERT para crear una unica uniad de trabajo.

In [7]:
BEGIN TRY
	BEGIN TRANSACTION
		INSERT INTO dbo.SimpleOrders(custid, empid, orderdate) VALUES (68,9,'2006-07-12');
		INSERT INTO dbo.SimpleOrderDetails(orderid,productid,unitprice,qty) VALUES (1, 2,15.20,20);
	COMMIT TRANSACTION
END TRY
BEGIN CATCH
	SELECT ERROR_NUMBER() AS ErrNum, ERROR_MESSAGE() AS ErrMsg;
	ROLLBACK TRANSACTION
END CATCH;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.140

2.3. Verificar el resultado.

In [8]:
SELECT  orderid, custid, empid, orderdate
FROM dbo.SimpleOrders;
SELECT  orderid, productid, unitprice, qty
FROM dbo.SimpleOrderDetails;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.266

orderid,custid,empid,orderdate
1,68,9,2006-07-12 00:00:00.000


orderid,productid,unitprice,qty
1,2,15.2000,20


2.4. Eliminamos todas las filas creadas anteriormente.

In [9]:
DELETE FROM dbo.SimpleOrderDetails;
GO
DELETE FROM dbo.SimpleOrders;
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.381

2.5. Ejecutamos nuevamente pero ahora con errores para ver el manejo de ROLLBACK.

In [14]:
BEGIN TRY
	BEGIN TRANSACTION
		INSERT INTO dbo.SimpleOrders(custid, empid, orderdate) VALUES (68,9,'2006-07-15');
		INSERT INTO dbo.SimpleOrderDetails(orderid,productid,unitprice,qty) VALUES (99, 2,15.20,20);
	COMMIT TRANSACTION
END TRY
BEGIN CATCH
	SELECT ERROR_NUMBER() AS ErrNum, ERROR_MESSAGE() AS ErrMsg;
	ROLLBACK TRANSACTION
END CATCH;

(1 row affected)

(0 rows affected)

(1 row affected)

Total execution time: 00:00:00.319

ErrNum,ErrMsg
547,"The INSERT statement conflicted with the FOREIGN KEY constraint ""FK__SimpleOrd__order__6E01572D"". The conflict occurred in database ""TSQL"", table ""dbo.SimpleOrders"", column 'orderid'."


2.6. Verificamos que ningún resultado parcial permanece.

In [15]:
SELECT  orderid, custid, empid, orderdate
FROM dbo.SimpleOrders;
SELECT  orderid, productid, unitprice, qty
FROM dbo.SimpleOrderDetails;

(1 row affected)

(0 rows affected)

Total execution time: 00:00:00.182

orderid,custid,empid,orderdate
1,68,9,2006-07-15 00:00:00.000


orderid,productid,unitprice,qty


2.7. Limpiamos los cambios.

In [16]:
IF OBJECT_ID('dbo.SimpleOrderDetails','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrderDetails;
IF OBJECT_ID('dbo.SimpleOrders','U') IS NOT NULL
	DROP TABLE dbo.SimpleOrders;

Commands completed successfully.

Total execution time: 00:00:00.186